In [1]:
# Import the required libraries
import pandas as pd

In [12]:
# Create pandas dataframe and query the dataset
df = pd.read_csv('tmp/housing_data.csv', header=None)
labels = ['title', 'price', 'location', 'bathrooms', 'bedrooms', 'size_msq']

rentals_df = pd.DataFrame(df.values, columns=labels)
rentals_df.head()

,title,price,location,bathrooms,bedrooms,size_msq
0,4 Bedroom Apartment / Flat,KSh 350 000,"Riverside Dr Nairobi, Riverside, Nairobi",4.0,4.0,NaN
1,2 Bedroom Apartment / Flat,KSh 150 000,"Parklands Nairobi, Parklands, Nairobi",2.5,2.0,NaN
2,Commercial Property,KSh 315 000,"Raphta Rd Muthangari, Westlands, Nairobi",NaN,NaN,420 m²
3,Apartment / Flat,KSh 26 000,"Kileleshwa, Kileleshwa, Nairobi",NaN,NaN,NaN
4,1 Bedroom Apartment / Flat,KSh 180 000,"Kilimani, Nairobi",1.0,1.0,NaN


In [14]:
# Data exploration
rentals_df.shape

(242403, 6)

In [16]:
rentals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242403 entries, 0 to 242402
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      242403 non-null  object
 1   price      242403 non-null  object
 2   location   217439 non-null  object
 3   bathrooms  225826 non-null  object
 4   bedrooms   227241 non-null  object
 5   size_msq   74025 non-null   object
dtypes: object(6)
memory usage: 11.1+ MB


In [1]:
rentals_df['size_msq'].unique()

NameError: name 'rentals_df' is not defined